In [ ]:
import mysql
import mysql.connector as connection
import pandas as pd
from google.cloud import storage

def export_table_to_gcs(request):
    
    request_args = request.args
    table_name = request_args['table_name']
    
    # MySQL database connection
    mydb = connection.connect(
        host="34.16.84.34",
        user="root",
        passwd="123Qaz456$$D8",
        database="classicmodels"
    )
    
    # SQL query to select data from the specified table
    query = f"SELECT * FROM {table_name}"
    
    # Read data from MySQL into a DataFrame
    result_df = pd.read_sql(query, mydb)
    
    # Close the MySQL database connection
    mydb.close()
    
    # Create a client for Google Cloud Storage
    storage_client = storage.Client()
    
    # Get the specified GCS bucket
    bucket = storage_client.bucket("gcp_api_etl_rpoject")
    
    # Define the path to the CSV file in GCS
    file_name = f"{table_name}.csv"
    blob = bucket.blob(file_name)
    
    # Convert DataFrame to CSV and upload to GCS
    result_df.to_csv(file_name, index=False)
    blob.upload_from_filename(file_name)
    
    return f"Data from {table_name} exported to gcp_api_etl_rpoject/{file_name}"